### Imports

In [14]:
import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split

In [15]:
print(tf.config.list_physical_devices('GPU'))

[]


First we have to transform our labels data set into a csv file

In [16]:
# Iterate over each label file in the labels folder
def create_csv_annotations(images_folder, labels_folder, name):
    annotations = []
    image_width = 224
    image_height = 224

    for label_file in os.listdir(labels_folder):
        if label_file.endswith('.txt'):
            with open(os.path.join(labels_folder, label_file), 'r') as f:
                lines = f.readlines()
            
            image_name = os.path.splitext(label_file)[0] + '.jpg'
            image_path = os.path.join(images_folder, image_name)
            
            for line in lines:
                class_label, x_center, y_center, width, height = map(float, line.split())
                x_min = (x_center - width / 2)
                y_min = (y_center - height / 2)
                x_max = (x_center + width / 2)
                y_max = (y_center + height / 2)
                
                annotations.append([image_path, x_min, y_min, x_max, y_max, image_width, image_height, class_label])

        # Here we create a DataFrame from annotations list and then we convert the df into a csv file
        df = pd.DataFrame(annotations, columns=['img_path', 'xmin', 'ymin', 'xmax', 'ymax', 'width', 'height', 'label'])
        df.to_csv(name, index=False)

In [17]:
create_csv_annotations('images/train', 'labels/train', 'annotations_train.csv')

In [18]:
create_csv_annotations('images/test', 'labels/test', 'annotations_test.csv')

In [19]:
create_csv_annotations('images/val', 'labels/val', 'annotations_val.csv')

[Tensor flow intro -> Why sequential](https://towardsdatascience.com/a-comprehensive-introduction-to-tensorflows-sequential-api-and-model-for-deep-learning-c5e31aee49fa#:~:text=The%20sequential%20model%20allows%20us,for%20building%20deep%20learning%20models.)

[Input and output shapes for CNN](https://towardsdatascience.com/understanding-input-and-output-shapes-in-convolution-network-keras-f143923d56ca)

[Basics of the R-CNN model](https://towardsdatascience.com/object-detection-explained-r-cnn-a6c813937a76)

In [20]:
# Load annotations from CSV
train_annotations = pd.read_csv('annotations_train.csv')
val_annotations = pd.read_csv('annotations_val.csv')
test_annotations = pd.read_csv('annotations_test.csv')

#train_annotations, val_annotations = train_test_split(annotations, test_size=0.2, random_state=42)


input_shape = (224, 224, 3)  # height, width, depth (this is the # of color channels RGB = 3)

# Function to preprocess image and annotations -> this is because 
# the annotations are still not in the format required for TF
def preprocess_data(annotation):
    image = load_img(annotation['img_path'], target_size=(input_shape[0], input_shape[1]))
    image_array = img_to_array(image)
    image_array /= 255.0
    bbox = [annotation['xmin'], annotation['ymin'], annotation['xmax'], annotation['ymax']]
    label = annotation['label']
    return image_array, bbox, label

train_data = train_annotations.apply(preprocess_data, axis=1)
val_data = val_annotations.apply(preprocess_data, axis=1)
test_data = test_annotations.apply(preprocess_data, axis=1)

# Convert preprocessed data into arrays -> this is the format needed for TF
X_train, y_train_bbox, y_train_label = zip(*train_data)
X_val, y_val_bbox, y_val_label = zip(*val_data)
X_test, y_test_bbox, y_test_label = zip(*test_data)

# Convert lists to numpy arrays
X_train = tf.convert_to_tensor(X_train)
y_train_bbox = tf.convert_to_tensor(y_train_bbox)
y_train_label = tf.convert_to_tensor(y_train_label)
X_val = tf.convert_to_tensor(X_val)
y_val_bbox = tf.convert_to_tensor(y_val_bbox)
y_val_label = tf.convert_to_tensor(y_val_label)
X_test = tf.convert_to_tensor(X_test)
y_test_bbox = tf.convert_to_tensor(y_test_bbox)
y_test_label = tf.convert_to_tensor(y_test_label)


In [21]:
len(X_train)

2776

In [33]:
X_train.shape

TensorShape([2776, 224, 224, 3])

In [37]:
y_test_label.shape

TensorShape([859])

In [31]:
y_train_label

<tf.Tensor: shape=(2776,), dtype=float32, numpy=array([3., 3., 3., ..., 2., 1., 1.], dtype=float32)>

In [22]:
len(X_val)

424

In [23]:
len(X_test)

859

In [30]:
y_test_label

<tf.Tensor: shape=(859,), dtype=float32, numpy=
array([3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 2., 2., 4., 1., 1., 4.,
       1., 1., 4., 1., 2., 3., 4., 4., 3., 2., 2., 2., 2., 2., 2., 2., 2.,
       3., 2., 4., 4., 2., 2., 2., 4., 4., 4., 4., 1., 1., 1., 1., 1., 1.,
       1., 3., 3., 3., 2., 2., 2., 1., 1., 4., 4., 3., 3., 3., 2., 2., 4.,
       3., 3., 3., 3., 3., 3., 3., 3., 3., 3., 2., 2., 3., 3., 1., 1., 2.,
       2., 3., 3., 3., 2., 3., 3., 4., 4., 4., 1., 1., 1., 4., 1., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 3., 3., 2., 2., 3., 3., 2., 2., 3., 3.,
       3., 3., 3., 4., 1., 1., 4., 4., 0., 1., 4., 1., 1., 0., 3., 2., 2.,
       3., 2., 3., 2., 1., 0., 0., 1., 0., 1., 1., 0., 2., 3., 3., 2., 2.,
       2., 4., 4., 2., 3., 1., 2., 3., 3., 3., 3., 3., 3., 3., 3., 2., 2.,
       3., 3., 3., 3., 3., 3., 3., 3., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1

## First model

In [ ]:
# This is the RCNN model, this is just base model for testing
def create_rcnn_model(input_shape, num_classes):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),#batch_input_shape (if you wanted to give the batch_size)
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.Flatten(), # Here we are basically changing the 4D output of the CNN to 2D so that we can use Dense
        layers.Dense(64, activation='relu'),
        layers.Dense(num_classes, activation='softmax', name='classifier_output')
    ])
    return model

# Number of classes
num_classes = train_annotations['label'].nunique()

# Create an instance of the R-CNN model
rcnn_model = create_rcnn_model(input_shape, num_classes)

# Compile the model with appropriate losses and metrics
rcnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
rcnn_model.fit(X_train, y_train_label, validation_data=(X_test, y_test_label), epochs=10)

## RCNN With MovileNetV2

In [24]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam

In [38]:
def create_rcnn_MovileNetV2(input_shape, num_classes):
    
    base_model = MobileNetV2(input_shape=input_shape, include_top=False, weights='imagenet')
    
    for layer in base_model.layers[-20:]:
        layer.trainable = True
    
    # Additional convolutional layers with reduced kernel size
    conv_layers = models.Sequential([ #Without the padding / strides I get dimensionality errors :(
        layers.Conv2D(32, (3, 3), activation='relu', padding='same'),  # Add padding to maintain spatial dimensions
        layers.MaxPooling2D((2, 2), strides=(1, 1)),  # Reduce the pooling stride
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),  # Add padding to maintain spatial dimensions
        layers.MaxPooling2D((2, 2), strides=(1, 1)),  # Reduce the pooling stride
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),  # Add padding to maintain spatial dimensions
    ])
    
    # R-CNN top layers
    top_layers = models.Sequential([
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(num_classes, activation='softmax', name='classifier_output')
    ])
    
    # Combine the base ResNet model, additional convolutional layers, and top layers
    model = models.Sequential([
        base_model,
        conv_layers,
        top_layers
    ])
    
    return model

# Number of classes
num_classes = train_annotations['label'].nunique()

input_shape = (224, 224, 3)  # 224 is the one used by ResNet
rcnn_resnet_model = create_rcnn_MovileNetV2(input_shape, num_classes)

rcnn_resnet_model.compile(optimizer=Adam(lr=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

rcnn_resnet_model.fit(X_train, y_train_label, validation_data=(X_test, y_test_label), epochs=15)


Epoch 1/15
87/87 [==============================] - 277s 3s/step - loss: 0.7821 - accuracy: 0.7003 - val_loss: 1.1293 - val_accuracy: 0.6752
Epoch 2/15
87/87 [==============================] - 257s 3s/step - loss: 0.6065 - accuracy: 0.7720 - val_loss: 0.9024 - val_accuracy: 0.6822
Epoch 3/15
87/87 [==============================] - 257s 3s/step - loss: 0.6084 - accuracy: 0.7622 - val_loss: 3.2185 - val_accuracy: 0.1281
Epoch 4/15
87/87 [==============================] - 257s 3s/step - loss: 0.5777 - accuracy: 0.7741 - val_loss: 1.2873 - val_accuracy: 0.2352
Epoch 5/15
87/87 [==============================] - 260s 3s/step - loss: 0.5349 - accuracy: 0.7878 - val_loss: 1.9865 - val_accuracy: 0.2421
Epoch 6/15
87/87 [==============================] - 257s 3s/step - loss: 0.5108 - accuracy: 0.7947 - val_loss: 3.5161 - val_accuracy: 0.3725
Epoch 7/15
87/87 [==============================] - 256s 3s/step - loss: 0.5183 - accuracy: 0.7918 - val_loss: 0.9597 - val_accuracy: 0.6694
Epoch 8/15
87

In [40]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

y_pred = np.argmax(rcnn_resnet_model.predict(X_val), axis=1)

y_val_label = np.array(y_val_label, dtype=int)
y_pred = np.array(y_pred, dtype=int)

target_names = ['Vehiculos', 'Construcciones', 'Vias', 'Rios', 'Mineria']  # Get unique class labels
print(classification_report(y_val_label, y_pred, target_names=target_names))

conf_matrix = confusion_matrix(y_val_label, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

14/14 [==============================] - 7s 525ms/step
                precision    recall  f1-score   support

     Vehiculos       0.00      0.00      0.00         3
Construcciones       0.80      0.93      0.86       254
          Vias       0.73      0.39      0.51        98
          Rios       0.67      0.83      0.74        60
       Mineria       0.00      0.00      0.00         9

      accuracy                           0.77       424
     macro avg       0.44      0.43      0.42       424
  weighted avg       0.74      0.77      0.74       424

Confusion Matrix:
[[  0   0   0   3   0]
 [  0 237   5  12   0]
 [  0  55  38   5   0]
 [  0   3   7  50   0]
 [  0   2   2   5   0]]


c:\Users\juank\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\juank\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\juank\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

## Resnet152

In [25]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet152V2
from tensorflow.keras import layers, models

In [41]:
def create_resnet152_model(input_shape, num_classes):
    # Load pre-trained ResNet152V2 model without the top classification layer
    base_model = ResNet152V2(weights='imagenet', include_top=False, input_shape=input_shape)

    # Freeze the weights of the pre-trained layers
    for layer in base_model.layers:
        layer.trainable = False

    # Add custom classification head
    top_layers = models.Sequential([
        layers.GlobalAveragePooling2D(),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])

    # Combine the base model with custom classification head
    model = models.Sequential([
        base_model,
        top_layers
    ])

    return model

# Define input shape and number of classes
input_shape = (224, 224, 3)  # Input shape for ResNet152V2
num_classes = train_annotations['label'].nunique()  # Example number of output classes

# Create the ResNet152 model
resnet152_model = create_resnet152_model(input_shape, num_classes)

# Compile the model
resnet152_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

resnet152_model.fit(X_train, y_train_label, validation_data=(X_test, y_test_label), epochs=10, batch_size=32)

# Print model summary
resnet152_model.summary()

Epoch 1/10
87/87 [==============================] - 392s 4s/step - loss: 0.8008 - accuracy: 0.7082 - val_loss: 0.5171 - val_accuracy: 0.7939
Epoch 2/10
87/87 [==============================] - 369s 4s/step - loss: 0.5806 - accuracy: 0.7677 - val_loss: 0.4816 - val_accuracy: 0.8044
Epoch 3/10
87/87 [==============================] - 373s 4s/step - loss: 0.5310 - accuracy: 0.7914 - val_loss: 0.5054 - val_accuracy: 0.8079
Epoch 4/10
87/87 [==============================] - 373s 4s/step - loss: 0.5063 - accuracy: 0.7968 - val_loss: 0.4566 - val_accuracy: 0.8091
Epoch 5/10
87/87 [==============================] - 371s 4s/step - loss: 0.4873 - accuracy: 0.7979 - val_loss: 0.4525 - val_accuracy: 0.8102
Epoch 6/10
87/87 [==============================] - 372s 4s/step - loss: 0.4576 - accuracy: 0.8030 - val_loss: 0.4564 - val_accuracy: 0.7986
Epoch 7/10
87/87 [==============================] - 371s 4s/step - loss: 0.4548 - accuracy: 0.8098 - val_loss: 0.4470 - val_accuracy: 0.8114
Epoch 8/10
87

In [42]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

y_pred = np.argmax(resnet152_model.predict(X_val), axis=1)

y_val_label = np.array(y_val_label, dtype=int)
y_pred = np.array(y_pred, dtype=int)

target_names = ['Vehiculos', 'Construcciones', 'Vias', 'Rios', 'Mineria']  # Get unique class labels
print(classification_report(y_val_label, y_pred, target_names=target_names))

conf_matrix = confusion_matrix(y_val_label, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

14/14 [==============================] - 46s 3s/step
                precision    recall  f1-score   support

     Vehiculos       0.00      0.00      0.00         3
Construcciones       0.82      1.00      0.90       254
          Vias       0.86      0.51      0.64        98
          Rios       0.98      0.85      0.91        60
       Mineria       0.40      0.22      0.29         9

      accuracy                           0.84       424
     macro avg       0.61      0.52      0.55       424
  weighted avg       0.84      0.84      0.82       424

Confusion Matrix:
[[  0   3   0   0   0]
 [  0 253   0   0   1]
 [  0  46  50   1   1]
 [  0   0   8  51   1]
 [  0   7   0   0   2]]


c:\Users\juank\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\juank\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\juank\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo